In [2]:
import pandas as pd
import numpy as np
import sklearn

In [75]:
data = pd.read_csv('checkins.dat', sep='|', skipinitialspace=True).dropna()

In [56]:
data.shape

(396634, 6)

In [92]:
columns_names = data.columns    # ['latitude      ', 'latitude      ']
mapper = {x: x.strip() for x in columns_names}
print(mapper)

needed_users = data.head(100000).rename(columns=mapper)[['latitude', 'longitude']]
centers = [((33.751277, -118.188740), 'Los Angeles'),
((25.867736, -80.324116), 'Miami'),
((51.503016, -0.075479), 'London'),
((52.378894, 4.885084), 'Amsterdam'),
((39.366487, 117.036146), 'Beijing'),
((-33.868457, 151.205134), 'Sydney')]

needed_users

{'id    ': 'id', 'user_id ': 'user_id', 'venue_id ': 'venue_id', 'latitude      ': 'latitude', 'longitude     ': 'longitude', 'created_at      ': 'created_at'}


,latitude,longitude
1,38.895112,-77.036366
3,33.800745,-84.410520
7,45.523452,-122.676207
9,40.764462,-111.904565
10,33.448377,-112.074037
...,...,...
233788,33.575000,-117.725556
233789,37.629349,-122.400087
233793,29.762884,-95.383061
233797,32.802955,-96.769923


In [93]:
import sklearn.cluster

In [94]:
clusterizer = sklearn.cluster.MeanShift(bandwidth=0.1)

In [165]:
pred = clusterizer.fit_predict(needed_users)

In [184]:
clusterizer.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [187]:
import collections
counter = collections.Counter(pred)

filtered = dict(filter(lambda obj: obj[1] > 15, counter.items()))
print(len(filtered))

keys = filtered.keys()

indexes_of_filtered_centers = []
for x in keys:
    indexes_of_filtered_centers.append([x * 2, x * 2 + 1])

centers_coordinates = np.take(clusterizer.cluster_centers_, indexes_of_filtered_centers)

592


In [222]:
from operator import itemgetter

distances = []
for coord in centers_coordinates:
    dist_to_point = []
    for office in centers:
        dist_to_point.append((office[0][0] - coord[0]) ** 2 + (office[0][1] - coord[1]) ** 2)
        
    distances.append(min(*dist_to_point))
    
distances_with_index = list(map(lambda x: (x[0], x[1]), enumerate(distances)))
lowest_index = sorted(distances_with_index, key=itemgetter(1))[0][0]

centers_coordinates[lowest_index]

array([-33.86063043, 151.20477593])